In [ ]:
# 1. 시스템 업데이트 및 언어 관련 패키지 설치
# (실행 시 시간이 좀 걸릴 수 있어요!)
!sudo apt-get update -qq
!sudo apt-get install locales -qq

# 2. 한국어 (ko_KR.UTF-8) locale 생성
# 이 단계에서 오류가 나지 않아야 해요!
!sudo locale-gen ko_KR.UTF-8

# 3. 환경 변수 설정
# 파이썬 코드 안에서 실행합니다.
import os
os.environ['LANG'] = 'ko_KR.UTF-8'
os.environ['LC_ALL'] = 'ko_KR.UTF-8'
os.environ['LC_CTYPE'] = 'ko_KR.UTF-8'
os.environ['LANGUAGE'] = 'ko_KR.UTF-8'

# 4. 런타임 다시 시작 (!!!! 아주 중요합니다 !!!!)
# 이 셀을 실행한 후에는 반드시 콜랩 메뉴에서 런타임을 재시작해야 해요.
# 메뉴: "런타임(Runtime)" -> "런타임 다시 시작(Restart runtime)" 클릭!
# 재시작 후에는 이 위의 코드 셀들을 다시 실행할 필요 없어요.
# 바로 다음 단계로 넘어가시면 됩니다.

# 5. (선택 사항) 설정 확인 - 런타임 재시작 후 이 셀을 실행해보세요.
# 'ko_KR.UTF-8' 관련 내용이 보이면 성공!
# !locale

런타임 다시 시작

In [ ]:
# 나눔 폰트 설치 (Colab에서 한글 표시를 위해 가장 많이 사용돼요)
!sudo apt-get install -y fonts-nanum > /dev/null 2>&1
!sudo fc-cache -fv > /dev/null 2>&1

# Matplotlib 등에서 한글 폰트 설정을 위한 코드 (streamlit과는 직접 관련 없을 수도 있지만,
# 만약을 위해 환경 준비 차원에서 실행해주세요)
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

# 설치된 폰트 경로 확인
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔바른고딕 예시
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    plt.rc('font', family='NanumBarunGothic')
    plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
    print("한글 폰트 설정 완료: NanumBarunGothic")
else:
    print(f"Warning: 폰트 파일이 없습니다: {font_path}")

# (선택 사항) 시스템에 설치된 폰트 목록 확인
# [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name]

In [ ]:
# 환경 점검 및 기본 설정
import torch                      # 딥러닝 프레임워크 PyTorch
import torch.nn as nn             # 신경망 모듈
import torch.optim as optim       # 최적화 알고리즘
from torchvision import datasets, transforms  # 데이터셋/전처리
from torch.utils.data import DataLoader       # 데이터 로더
import matplotlib.pyplot as plt    # 시각화
import numpy as np                 # 수치 연산
import random                      # 시드 고정

print('PyTorch  :', torch.__version__)  # 파이토치 버전 출력
import torchvision
print('TorchVision :', torchvision.__version__)  # 토치비전 버전 출력

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부
print('Device:', device)                                  # 디바이스 출력

In [ ]:
# 재현성 확보 및 하이퍼파라미터
SEED = 0                           # 임의성 제어용 시드
random.seed(SEED)                  # 파이썬 시드 고정
np.random.seed(SEED)               # 넘파이 시드 고정
torch.manual_seed(SEED)            # 파이토치 CPU 시드 고정
if torch.cuda.is_available():      # GPU 사용 시
    torch.cuda.manual_seed_all(SEED)  # 모든 GPU 시드 고정

BATCH_SIZE = 128                   # 배치 크기
EPOCHS = 3                         # 학습 에폭(데모용)
LR = 2e-3                          # 학습률
NUM_WORKERS = 2                    # DataLoader 병렬 워커 수

In [ ]:
# SVHN 데이터셋 로드/전처리
# SVHN 평균/표준편차 (경험치)
mean = (0.4377, 0.4438, 0.4728)    # 채널별 평균
std  = (0.1980, 0.2010, 0.1970)    # 채널별 표준편차

# 학습/테스트 공통 정규화
train_tf = transforms.Compose([
    transforms.ToTensor(),         # 텐서 변환
    transforms.Normalize(mean, std) # 정규화
])
test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# SVHN 다운로드/구성 (CIFAR-10 금지 조건을 충족하기 위해 SVHN 사용)
train_ds = datasets.SVHN(root='/content/data_svhn', split='train', download=True, transform=train_tf)
test_ds  = datasets.SVHN(root='/content/data_svhn', split='test',  download=True, transform=test_tf)

# 데이터로더 구성
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=256,        shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print('학습 샘플 수:', len(train_ds))  # 학습 데이터 개수
print('테스트 샘플 수:', len(test_ds)) # 테스트 데이터 개수

In [ ]:
# 데이터 미리보기
# 배치 하나를 얻어 이미지/레이블 확인
images, labels = next(iter(train_loader))      # 첫 미니배치
print('배치 이미지 shape:', images.shape)      # (B, C, H, W)
print('배치 레이블 shape:', labels.shape)      # (B,)

# 16개 샘플 시각화
grid = 16                                      # 표시 개수
plt.figure(figsize=(8,8))                      # 도화지 크기
for i in range(grid):                          # 16개 루프
    plt.subplot(4,4,i+1)                       # 4x4 서브플롯
    img = images[i].permute(1,2,0).cpu().numpy()     # (H,W,C)로 변환
    img = (img * np.array(std) + np.array(mean))     # 정규화 역변환
    img = np.clip(img, 0, 1)                         # 0~1로 클리핑
    plt.imshow(img)                                   # 이미지 표시
    plt.title(int(labels[i]))                         # 레이블 표시
    plt.axis('off')                                   # 축 숨김
plt.tight_layout()                                    # 레이아웃 정리
plt.show()                                            # 출력

In [ ]:
# 모델 #1: nn.Sequential로 CNN 설계
# Feature Extractor (특징추출) 파트
seq_feature_extractor = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 3->32 채널, 3x3 필터
    nn.ReLU(),                                   # 비선형 활성화
    nn.MaxPool2d(2),                              # 공간 크기 절반

    nn.Conv2d(32, 64, kernel_size=3, padding=1), # 32->64 채널
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),# 64->128 채널
    nn.ReLU(),
    nn.MaxPool2d(2)                               # 32->16->8->4
)
# Classifier (분류기) 파트
seq_classifier = nn.Sequential(
    nn.Flatten(),                                 # (B,128,4,4)->(B,2048)
    nn.Linear(128*4*4, 256),                      # 완전연결층
    nn.ReLU(),                                    # 활성화
    nn.Linear(256, 10)                            # SVHN 10 클래스
)
# 전체 모델 결합
seq_model = nn.Sequential(seq_feature_extractor, seq_classifier).to(device)

# 더미 입력으로 shape 확인
dummy = torch.randn(1, 3, 32, 32).to(device)     # 가짜 이미지 (배치, 채널수, 높이, 너비)
out = seq_model(dummy)                            # 순전파
print('Sequential 출력 shape:', out.shape)        # (1,10) 1이 가리키는 것은? Batch

# 파라미터 수 계산 함수
def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

print('학습 가능한 파라미터 수:', count_params(seq_model))  # 파라미터 총계

In [ ]:
# 모델 #2: 클래스형 CNN (forward 명확/확장 용이)
class ClassCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 특징추출부
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        # 분류기
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*4*4, 256), nn.ReLU(),
            nn.Linear(256, 10)
        )
        # He(kaiming) 초기화로 안정적 학습 시작
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)     # 특징 추출
        x = self.classifier(x)   # 분류
        return x

cls_model = ClassCNN().to(device)                 # 모델 인스턴스 생성
print('ClassCNN 파라미터 수:', count_params(cls_model))  # 파라미터 수 출력

# 레이어별 출력 shape 추적(후크)
shapes = []                                       # shape 저장 리스트
def hook(m, i, o):
    if isinstance(o, torch.Tensor):
        shapes.append(tuple(o.shape))             # 출력 shape 기록

hooks = []
for layer in cls_model.features:                  # 특징추출부 레이어 순회
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)):
        hooks.append(layer.register_forward_hook(hook))  # 후크 등록
_ = cls_model(dummy)                              # 더미 순전파로 후크 실행
for h in hooks: h.remove()                        # 후크 해제
print('features 출력 shapes:', shapes)            # 레이어별 shape 출력


In [ ]:
# 학습/평가 유틸 함수
def accuracy(outputs, targets):
    preds = outputs.argmax(dim=1)           # 예측 클래스
    return (preds == targets).float().mean().item()  # 정확도

# 1 에폭 학습
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()                           # 학습 모드
    tot_loss, tot_acc, tot_cnt = 0.0, 0.0, 0
    for x, y in loader:                     # 미니배치 반복
        x, y = x.to(device), y.to(device)   # 디바이스 이동

        optimizer.zero_grad()               # 기울기 초기화
        out = model(x)                      # 순전파
        loss = criterion(out, y)            # 손실 계산
        loss.backward()                     # 역전파
        optimizer.step()                    # 파라미터 갱신

        tot_loss += loss.item() * y.size(0) # 손실 누적
        tot_acc  += (out.argmax(1) == y).float().sum().item() # 정답수 누적
        tot_cnt  += y.size(0)               # 샘플 수 누적
    return tot_loss/tot_cnt, tot_acc/tot_cnt  # 평균 손실/정확도

# 평가
def evaluate(model, loader, criterion):
    model.eval()                            # 평가 모드
    tot_loss, tot_acc, tot_cnt = 0.0, 0.0, 0
    with torch.no_grad():                   # 기울기 미계산
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            tot_loss += loss.item() * y.size(0)
            tot_acc  += (out.argmax(1) == y).float().sum().item()
            tot_cnt  += y.size(0)
    return tot_loss/tot_cnt, tot_acc/tot_cnt

In [ ]:
# Sequential 모델 학습/그래프
criterion = nn.CrossEntropyLoss()                # 다중분류 손실
optimizer = optim.AdamW(seq_model.parameters(), lr=LR)  # AdamW 최적화기

tr_hist, te_hist = [], []                        # 기록용 리스트
for ep in range(1, EPOCHS+1):                    # 에폭 반복
    tr_loss, tr_acc = train_one_epoch(seq_model, train_loader, optimizer, criterion)
    te_loss, te_acc = evaluate(seq_model, test_loader, criterion)
    tr_hist.append((tr_loss, tr_acc))
    te_hist.append((te_loss, te_acc))
    print(f"[Sequential] Epoch {ep}/{EPOCHS} | train {tr_acc:.3f}/{tr_loss:.3f} | test {te_acc:.3f}/{te_loss:.3f}")

# 정확도 곡선
plt.figure(); plt.plot([a for _,a in tr_hist], label='train acc'); plt.plot([a for _,a in te_hist], label='test acc')
plt.legend(); plt.title('정확도 추세(Sequential)'); plt.show()

# 손실 곡선
plt.figure(); plt.plot([l for l,_ in tr_hist], label='train loss'); plt.plot([l for l,_ in te_hist], label='test loss')
plt.legend(); plt.title('손실 추세(Sequential)'); plt.show()

In [ ]:
# 하이퍼파라미터 간단 실험
class SmallExp(nn.Module):
    def __init__(self, ch1=16, ch2=32, k=3, stride=1):
        super().__init__()
        pad = k//2                                 # 출력 사이즈 보존용 패딩
        self.net = nn.Sequential(
            nn.Conv2d(3,  ch1, k, stride=stride, padding=pad), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(ch1, ch2, k, stride=1,      padding=pad), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(ch2*8*8, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self,x): return self.net(x)        # 순전파 정의

def quick_eval(ch1, ch2, k, stride):
    m = SmallExp(ch1, ch2, k, stride).to(device)   # 모델 생성/이동
    opt = optim.AdamW(m.parameters(), lr=LR)       # 최적화기
    crit = nn.CrossEntropyLoss()                   # 손실함수
    for _ in range(2):                             # 데모용 2에폭만
        train_one_epoch(m, train_loader, opt, crit)
    _, acc = evaluate(m, test_loader, crit)        # 테스트 정확도
    return acc

settings = [                                      # 실험 설정들
    {'ch1':16,'ch2':32,'k':3,'stride':1},
    {'ch1':32,'ch2':64,'k':3,'stride':1},
    {'ch1':32,'ch2':64,'k':5,'stride':1}
]

results = []
for s in settings:                                 # 설정 반복
    acc = quick_eval(**s)                          # 설정 실행
    results.append((s, acc))                       # 결과 저장
    print('설정:', s, '| 테스트 정확도:', round(acc,4))

# 막대 그래프 표시
labels = [f"{r[0]['ch1']}/{r[0]['ch2']},k{r[0]['k']},s{r[0]['stride']}" for r in results]
vals   = [r[1] for r in results]
plt.figure(figsize=(8,3)); plt.bar(range(len(vals)), vals)
plt.xticks(range(len(vals)), labels, rotation=30, ha='right')
plt.title('하이퍼파라미터 변화에 따른 정확도(간이)')
plt.show()
